# Interactive notebook for composer recommendations

## Instructions

Replace 'Benjamin Britten' as composer_name below, with a composer of your choice, and click Cell > Run All.

Scroll down to the final cell in the notebook, which will contain some semantically-informed recommendations.

In [1]:
composer_name = 'Benjamin Britten'

In [2]:
embeddings_path = '../data/embeddings/composer-embeddings-c2v-dm-5000-10000.h5'

In [3]:
from difflib import SequenceMatcher
import glob
import re

import h5py
import numpy as np
import pandas as pd
from scipy.spatial import distance

/Users/sam/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
all_composers = [(i, *c) for i, c in enumerate(pd.read_csv('../data/composers.csv', sep='|').values)]

In [5]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def name_to_composer_id(name):
    composer = max(all_composers, key=lambda c: similar(c[1], name))
    composer_id = composer[0]
    print('You specified: {}'.format(name))
    print('Assuming you meant {}: born {}; died {}; {}'.format(composer[1], composer[2], composer[3], composer[-1]))
    return composer_id

In [6]:
def path_to_embedding(path):
    with h5py.File(path, 'r') as f:
        return f.get('doc_embeddings').value

In [7]:
embeddings = path_to_embedding(embeddings_path)

In [8]:
def self_similarity(embeddings):
    return distance.cdist(embeddings, embeddings, metric='cosine')

distances = self_similarity(embeddings)
closest = distances.argsort()

In [9]:
target_id = name_to_composer_id(composer_name)

print('You might like the following:')

for c_id in closest[target_id, 1:6]:
    composer = all_composers[c_id]
    
    _, name, *_, url = composer
    
    print('{}: {}'.format(name, url))

You specified: Benjamin Britten
Assuming you meant Benjamin Britten: born 1913; died 1976; https://en.wikipedia.org/wiki/Benjamin_Britten
You might like the following:
Arnold Bax: https://en.wikipedia.org/wiki/Arnold_Bax
Ralph Vaughan Williams: https://en.wikipedia.org/wiki/Ralph_Vaughan_Williams
William Walton: https://en.wikipedia.org/wiki/William_Walton
Gustav Holst: https://en.wikipedia.org/wiki/Gustav_Holst
Edward Elgar: https://en.wikipedia.org/wiki/Edward_Elgar
